# Detection with YOLO

In this section, we will begin exploring our detection pipeline using **YOLOv11** as the primary object detector.

1. Relabelling the drowning detection dataset for human detection task
2. A series of **visualizations** to understand how YOLO processes each image — including bounding box predictions, confidence scores, and the role of preprocessing.
3. Experimentation with **hyperparameter tuning** to adjust settings like learning rate, batch size, and optimizer behavior.
4. **Fine-tuning** the YOLO model on the drowning detection dataset to improve accuracy in real-world aquatic scenarios.

This serves as the foundation before integrating it with a secondary classifier for behavior prediction (e.g., swimming, treading water, drowning).


In [1]:
from dotenv import load_dotenv

load_dotenv()

import sys
import os

sys.path.append(os.getenv('SRC_DIR'))

from utils.system import display_system_info

display_system_info(markdown=True)


**Last Updated**: 2025-04-23 21:47:13

**Python Version**: 3.11.11  
**OS**: Windows 10.0.26100  
**Architecture**: 64bit  
**Hostname**: DESKTOP-42J9AQP  
**Processor**: Intel64 Family 6 Model 183 Stepping 1, GenuineIntel  
**RAM Size**: 63.85 GB  
  
        

In [2]:
from ultralytics import YOLO
import torch
import shutil

## Relabelling for YOLO Training

In [3]:
yolo_training_data_dir = os.getenv('YOLO_TRAINING_DATA_DIR')
raw_data_dir = os.getenv('RAW_DATA_DIR')
img_dir = os.getenv('IMG_DIR')
label_dir = os.getenv('LABEL_DIR')
train_dir = os.getenv('TRAIN_DIR')
val_dir = os.getenv('VAL_DIR')
yolo_config_dir = os.getenv('YOLO_CONFIG_DIR')

In [4]:
for split in [train_dir, val_dir]:
    img_src_dir = os.path.join(raw_data_dir, img_dir, split)
    label_src_dir = os.path.join(raw_data_dir, label_dir, split)

    img_dst_dir = os.path.join(yolo_training_data_dir, img_dir, split)
    label_dst_dir = os.path.join(yolo_training_data_dir, label_dir, split)

    os.makedirs(img_dst_dir, exist_ok=True)
    os.makedirs(label_dst_dir, exist_ok=True)

    for filename in os.listdir(img_src_dir):
        if filename.endswith('.jpg'):
            shutil.copy2(os.path.join(img_src_dir, filename), os.path.join(img_dst_dir, filename))

            label_filename = os.path.splitext(filename)[0] + '.txt'
            label_src_path = os.path.join(label_src_dir, label_filename)
            label_dst_path = os.path.join(label_dst_dir, label_filename)

            with open(label_src_path, 'r') as infile:
                lines = infile.readlines()
            
            with open(label_dst_path, 'w') as outfile:
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        newline = f"0 {' '.join(parts[1:])} \n"
                        outfile.write(newline)

## YOLO Fine-Tuning

In [5]:
yaml_content = (
f"""# Dataset paths
path: {yolo_training_data_dir}
train: {os.path.join(img_dir, train_dir)}
val: {os.path.join(img_dir, val_dir)}

# Number of classes
nc: 1
# Class names
names:
  0: person
""")

yaml_path = os.path.join(yolo_config_dir, "fine_tuning.yaml")
with open(yaml_path, "w") as f:
    f.write(yaml_content)
print(f"Created data configuration at {yaml_path}")

Created data configuration at C:\Users\PC\Downloads\JieShen\Drowning-Detection/config/detection/YOLO\fine_tuning.yaml


In [7]:
model = YOLO(os.path.join(os.getenv('YOLO_MODEL_DIR'), 'yolo11n.pt'))

results = model.train(
    data=yaml_path,
    project=os.getenv('YOLO_MODEL_DIR'),
    name='yolo11n-finetuned',
    exist_ok=True,
    fraction=1,
    val=True,
    plots=True,
    resume=False,
    # Training configurations
    epochs=50,
    patience=20,
    batch=16,
    imgsz=640,
    optimizer='auto',
    # Logging configurations
    save=True,
    save_period=1,
    cache=False,
    # Training device configurations
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=8,
    seed=0,
    # Algorithm configurations
    deterministic=True,
    # Augmentation configurations
    hsv_h=0.09,
    hsv_s=0.95,
    hsv_v=0.7,
    degrees=10.0,
    fliplr=0.5,
    scale=0.5,
    translate=0.1,
    dropout=0,
    mosaic=1
)

Ultralytics 8.3.114  Python-3.11.11 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
engine\trainer: task=detect, mode=train, model=C:\Users\PC\Downloads\JieShen\Drowning-Detection/models/detection/YOLO\yolo11n.pt, data=C:\Users\PC\Downloads\JieShen\Drowning-Detection/config/detection/YOLO\fine_tuning.yaml, epochs=50, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=0, workers=8, project=C:\Users\PC\Downloads\JieShen\Drowning-Detection/models/detection/YOLO, name=yolo11n-finetuned, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment

train: Scanning C:\Users\PC\Downloads\JieShen\Drowning-Detection\data\yolo_training\labels\train.cache... 7000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7000/7000 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.00.0 ms, read: 1940.6183.6 MB/s, size: 2040.0 KB)


val: Scanning C:\Users\PC\Downloads\JieShen\Drowning-Detection\data\yolo_training\labels\val.cache... 1572 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1572/1572 [00:00<?, ?it/s]


Plotting labels to C:\Users\PC\Downloads\JieShen\Drowning-Detection\models\detection\YOLO\yolo11n-finetuned\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to C:\Users\PC\Downloads\JieShen\Drowning-Detection\models\detection\YOLO\yolo11n-finetuned
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      2.35G     0.8899     0.3569     0.9517         11        640: 100%|██████████| 438/438 [00:38<00:00, 11.50it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:05<00:00,  8.77it/s]

                   all       1572       2317      0.995      0.998      0.994      0.789



50 epochs completed in 0.639 hours.
Optimizer stripped from C:\Users\PC\Downloads\JieShen\Drowning-Detection\models\detection\YOLO\yolo11n-finetuned\weights\last.pt, 5.4MB
Optimizer stripped from C:\Users\PC\Downloads\JieShen\Drowning-Detection\models\detection\YOLO\yolo11n-finetuned\weights\best.pt, 5.4MB

Validating C:\Users\PC\Downloads\JieShen\Drowning-Detection\models\detection\YOLO\yolo11n-finetuned\weights\best.pt...
Ultralytics 8.3.114  Python-3.11.11 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:06<00:00,  7.79it/s]


                   all       1572       2317      0.994      0.998      0.994      0.796
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to C:\Users\PC\Downloads\JieShen\Drowning-Detection\models\detection\YOLO\yolo11n-finetuned


In [ ]:
# !tensorboard --logdir {os.path.join(os.getenv('YOLO_MODEL_DIR'), 'yolo11n-finetuned')} --port 6006